In [95]:
import pandas as pd
from influxdb import InfluxDBClient
from influxdb import SeriesHelper
from influxdb import DataFrameClient
import datetime
import matplotlib.pyplot as plt

In [96]:
host = '183.136.205.102'
port = 38086
user = 'root'
password = 'root'
dbname = 'chq'

In [97]:
# myclient = InfluxDBClient(host, port, user, password, 'market')
# myclient.create_database('market')
# myclient.create_retention_policy('inf_policy', 'INF', 1, default=True)
# client = DataFrameClient(host, port, user, password, 'challenge')
# client.query('drop measurement "idx.positon"')

## 指数行情

In [98]:
market_dbname = 'market'
market_series_name = "index.day"
market_client = DataFrameClient(host, port, user, password, market_dbname)

### 更新指数行情

In [99]:
filename = "idx_market.csv"
def get_index_market_per_day(filename):
    idx_market = pd.read_csv(filename, dtype={'ticker':str, 'name': str}, parse_dates=['date'])
    idx_market = idx_market.drop('Unnamed: 0', axis=1)
    idx_market = idx_market.set_index('date')
    return idx_market

In [100]:
idx_market = get_index_market_per_day(filename)
idx_market.head()

,ticker,name,pre_close,open,low,high,close
date,,,,,,,
2017-10-10,000016,上证50,2692.0769,2693.2510,2671.9426,2711.3065,2695.5374
2017-10-11,000016,上证50,2695.5374,2694.4982,2692.7692,2717.3281,2707.5221
2017-10-12,000016,上证50,2707.5221,2706.4203,2702.3134,2724.8346,2718.3508
2017-10-13,000016,上证50,2718.3508,2716.0041,2711.9544,2722.2311,2713.3767
2017-10-16,000016,上证50,2713.3767,2719.3095,2715.0511,2741.0971,2726.6423


In [101]:
market_client.write_points(idx_market, market_series_name, tag_columns=['ticker', 'name'], field_columns=['pre_close', 'open', 'low', 'high', 'close'])

True

### 获取指数行情

In [102]:
df = market_client.query('select * from  "{0}"'.format(market_series_name))
market = df[series_name]
market.tail()

,close,high,low,name,open,pre_close,ticker
2017-10-17 00:00:00+00:00,6612.9428,6644.7522,6589.5417,中证500,6604.3300,6619.2056,000905
2017-10-17 00:00:00+00:00,2723.5345,2734.3072,2716.4042,上证50,2725.1997,2726.6423,000016
2017-10-18 00:00:00+00:00,6563.4369,6630.6685,6557.6601,中证500,6612.4885,6612.9428,000905
2017-10-18 00:00:00+00:00,2753.0105,2753.9229,2723.7626,上证50,2725.6666,2723.5345,000016
2017-10-18 00:00:00+00:00,3944.1626,3944.4106,3914.8152,沪深300,3914.8152,3913.0688,000300


### 更新仓位信息

In [103]:
files = ['1011', '1012', '1013', '1016', '1017', '1018'] #文件{%m%d}.csv
def get_positions(files):
    all_pos = pd.DataFrame()
    for file in files:
        pos = pd.read_csv('position/{0}.csv'.format(file), encoding='gbk')
        pos = pos.rename(columns={"Unnamed: 0": 'user'})
        pos = pos.dropna(how='all', axis=1).dropna(how='all', axis=0)
        pos['date'] = datetime.datetime.strptime('2017{0}'.format(file), "%Y%m%d")
        all_pos = pd.concat([all_pos, pos])
    all_pos = all_pos.fillna(0)
    all_pos = all_pos.set_index(['date', 'user'])
    all_pos = all_pos.stack()
    all_pos = all_pos.reset_index().set_index('date')
    all_pos.columns = ['user', 'name', 'position']
    return all_pos

In [104]:
new_pos = get_positions(files)

In [105]:
new_pos.tail()

,user,name,position
date,,,
2017-10-18,张金生,沪深300,0.0
2017-10-18,张金生,中证500,0.0
2017-10-18,肖鹏懿,上证50,0.1
2017-10-18,肖鹏懿,沪深300,0.0
2017-10-18,肖鹏懿,中证500,0.3


In [106]:
challenge_dbname = "challenge"
pos_series_name = "idx.position"
# myclient = InfluxDBClient(host, port, user, password, dbname)
# myclient.create_database(dbname)
# myclient.create_retention_policy('inf_policy', 'INF', 1, default=True)
challenge_client = DataFrameClient(host, port, user, password, challenge_dbname)

In [107]:
challenge_client.write_points(new_pos, pos_series_name, tag_columns=['user', 'name'], field_columns=['position'])

True

### 获取仓位信息

In [108]:
df = client.query('select * from  "{0}"'.format(pos_series_name))

In [109]:
all_pos = df[pos_series_name]

In [110]:
all_pos.tail()

,name,position,user
2017-10-18 00:00:00+00:00,沪深300,0.0,孙洋
2017-10-18 00:00:00+00:00,中证500,0.0,许小强
2017-10-18 00:00:00+00:00,中证500,0.0,张金生
2017-10-18 00:00:00+00:00,沪深300,0.1,严龙灿
2017-10-18 00:00:00+00:00,中证500,0.5,张超


### 计算净值

In [111]:
def cal_return(x):
    return ((x['close'] - x['open']) * x['position'] + (x['open'] - x['pre_close']) * x['pre_position'])  / x['pre_close'] 

In [112]:
dbname = "challenge"
challenge_client = DataFrameClient(host, port, user, password, dbname)

In [113]:
def cal_returns(position, market):
    return_series_name = "idx.return"
    for user in position.user.unique():
        for idx_name in position.name.unique():
            user_idx = position[(position['user']==user) & (position['name'] == idx_name)].copy()
            user_idx['pre_position'] = user_idx['position'].shift(1)
            user_idx = user_idx.fillna(0)
            market_idx = market[market['name']==idx_name]
            profit = pd.merge(user_idx, market_idx, left_index=True, right_index=True, on='name')
            profit['profit'] = profit.apply(cal_return, axis=1)
            profit['profit'] = profit['profit'] + 1
            profit['return'] = profit['profit'].cumprod()
            challenge_client.write_points(profit, return_series_name, tag_columns=['user', 'name', 'ticker'], field_columns=['profit', 'return'])

In [114]:
all_pos.tail()

,name,position,user
2017-10-18 00:00:00+00:00,沪深300,0.0,孙洋
2017-10-18 00:00:00+00:00,中证500,0.0,许小强
2017-10-18 00:00:00+00:00,中证500,0.0,张金生
2017-10-18 00:00:00+00:00,沪深300,0.1,严龙灿
2017-10-18 00:00:00+00:00,中证500,0.5,张超


In [115]:
market.tail()

,close,high,low,name,open,pre_close,ticker
2017-10-17 00:00:00+00:00,6612.9428,6644.7522,6589.5417,中证500,6604.3300,6619.2056,000905
2017-10-17 00:00:00+00:00,2723.5345,2734.3072,2716.4042,上证50,2725.1997,2726.6423,000016
2017-10-18 00:00:00+00:00,6563.4369,6630.6685,6557.6601,中证500,6612.4885,6612.9428,000905
2017-10-18 00:00:00+00:00,2753.0105,2753.9229,2723.7626,上证50,2725.6666,2723.5345,000016
2017-10-18 00:00:00+00:00,3944.1626,3944.4106,3914.8152,沪深300,3914.8152,3913.0688,000300


In [116]:
cal_returns(all_pos, market)

### 获取净值信息

In [117]:
return_series_name = "idx.return"
df = challenge_client.query('select * from  "{0}"'.format(return_series_name))
returns = df[return_series_name]
returns.tail()

,name,profit,return,ticker,user
2017-10-18 00:00:00+00:00,沪深300,1.000000,1.005035,000300,孙洋
2017-10-18 00:00:00+00:00,中证500,1.000000,1.001325,000905,许小强
2017-10-18 00:00:00+00:00,中证500,1.000000,0.996671,000905,张金生
2017-10-18 00:00:00+00:00,沪深300,1.000795,1.005951,000300,严龙灿
2017-10-18 00:00:00+00:00,中证500,0.996257,0.993702,000905,张超


In [118]:
for key, grp in returns.groupby(['name']):
    for key, grp1 in grp.groupby(['user']):
        grp1.plot()
    # plt.plot(grp['profit'], label=key)
#     grp['D'] = pd.rolling_mean(grp['B'], window=5)    
#     plt.plot(grp['D'], label='rolling ({k})'.format(k=key))
plt.legend(loc='best')    
plt.show

/root/.pyenv/versions/3.5.3/lib/python3.5/site-packages/matplotlib/pyplot.py:524: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  max_open_warning, RuntimeWarning)


<function matplotlib.pyplot.show>

In [119]:
returns

,name,profit,return,ticker,user
2017-09-26 00:00:00+00:00,上证50,1.001625,1.001625,000016,严龙灿
2017-09-26 00:00:00+00:00,沪深300,1.000000,1.000000,000300,陈红青
2017-09-26 00:00:00+00:00,上证50,1.001422,1.001422,000016,孙洋
2017-09-26 00:00:00+00:00,上证50,1.001625,1.001625,000016,朱奕杰
2017-09-26 00:00:00+00:00,中证500,1.000000,1.000000,000905,刘一奇
2017-09-26 00:00:00+00:00,沪深300,1.000000,1.000000,000300,张金生
2017-09-26 00:00:00+00:00,沪深300,1.000000,1.000000,000300,张超
2017-09-26 00:00:00+00:00,中证500,1.000561,1.000561,000905,严龙灿
2017-09-26 00:00:00+00:00,沪深300,1.000547,1.000547,000300,孙洋
2017-09-26 00:00:00+00:00,沪深300,1.000000,1.000000,000300,刘一奇
